#  Understanding and Using database.py

## 1. Introduction

#### Purpose of the Notebook

This notebook serves as a comprehensive guide for understanding and working with the `Database` class defined in `database.py`. It is designed to:

* Demonstrate how user-generated data (e.g., prompts, images, chats) are recorded and retrieved.
* Explain the relationships between users, chats, prompts, images, and metrics.
* Show how internal methods such as `save_prompt` and `save_image` automatically compute and store metrics like BERTScore and LPIPS.
* Provide reproducible examples that seed the database with realistic test data for analysis and development.

By the end of the notebook, you’ll understand both the structure and practical usage of the database system for AI-human interaction logging.

#### Overview of the Database Schema

The database can be modelled as a hierarchical system:

```
User
└── Chat
    └── Prompt
        ├── Image(s)
        └── Metrics (BERTScore, LPIPS, etc.)
```

Tables created:

* `users`: Stores user credentials (username/password pairs).
* `chats`: Represents a user session or thread of interaction. Each user can have multiple chats. Each chat contains a sequence of user prompts.
* `prompts`: Contains text prompts given by the user.
* `images`: Metadata (such as image paths) about the images generated based on prompts.
* `bertscore_metrics`: Measures semantic novelty of new prompts based on BERTScore (high Bert = more different).
* `lpips_metrics`: Measures perceptual image distance between images visa LPIPS (high LPIPS = more different).
* `guidance_metrics`: Logs prompt/image guidance values.
* `functionality_metrics`: Tracks use of AI suggestion and enhancement features.
* `prompt_word_metrics`: Extracts and stores relevant words and counts from prompts.

Each entity is tied together by unique IDs and foreign keys (user_id, chat_id, etc.), enabling complex queries and behavioral analysis.

#### Dependencies

The core system relies on the following Python libraries and modules:

| Dependency                               | Purpose                                                    |
| ---------------------------------------- | ---------------------------------------------------------- |
| `sqlite3`                                | Lightweight relational database engine                     |
| `pandas`                                 | For returning and manipulating query results as DataFrames |
| `os`, `json`                             | File path handling and serialization                       |
| `PIL` (Pillow) (via `PromptImage`)               |  For image loading, manipulation and saving.                                   |
| `modules.prompt`, `prompt_image`, `chat` | Custom data structures for records, they are the core data models.                         |
| `modules.metrics`                        | Provides metric functions like BERTScore and LPIPS (`get_or_compute_bertscore`, `get_or_compute_lpips`, `extract_relevant_words`)         |
| `create.py`                              | Contains raw SQL for initializing table schemas            |



## 2. Setup

In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'app')))

from db.database import Database
from modules.prompt import Prompt
from modules.prompt_image import PromptImage
from PIL import Image
import uuid


/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/scur0279/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/scur0279/MMA/.venv/lib64/python3.9/site-packages/lpips/weights/v0.1/alex.pth


## 3. Database Initialization

You can reset the database, clearing all tables. This is helpful for clean testing.

In [6]:
db = Database()
db.connect()
# db.reset_tables()  # Be careful with this in production!
print("[Database reset completed]")


[Database reset completed]


## 4. User Insertion and Fetching

Thes following methods handle basic user management:

* `insert_user(username, password)` adds a new user to the database. This is used during registration.

* `fetch_user_by_username(username)` retrieves a user's info. It's useful for login checks and looking up user data.

In production, passwords should be securely hashed, not stored as plain text.
Together, they support authentication and link users to chats, prompts, and images.

In [4]:
# Insert sample users
db.insert_user("user1", "pass1")
db.insert_user("user2", "pass2")

# Fetch users
print(db.fetch_all_users())
print(db.fetch_user_by_username("user1"))

Error inserting user: UNIQUE constraint failed: users.username
Error inserting user: UNIQUE constraint failed: users.username
   id username password
0   1    user1    pass1
1   2    user2    pass2
   id username password
0   1    user1    pass1


## 5. Creating a Chat and Prompts

As previously mentioned, the database is organized hierarchically: user → chat → prompt.

In addition, each user can have multiple chats (sessions), and each chat can contain multiple prompts (individual user inputs).

####  Prompt Metadata

Each prompt record captures rich context and behavior. The `prompts` table stores the following metadata:

| Field                 | Description                                          |
| --------------------- | ---------------------------------------------------- |
| `id`                  | Unique identifier for the prompt                     |
| `chat_id`             | Link to the chat this prompt belongs to              |
| `user_id`             | Link to the user who created it                      |
| `prompt`              | Raw user input text                                  |
| `depth`               | Position in the conversation thread (e.g., 1st, 2nd) |
| `used_suggestion`     | Whether an AI suggestion was used                    |
| `modified_suggestion` | Whether the suggestion was changed before use        |
| `suggestion_used`     | Text of the suggestion (if any)                      |
| `is_enhanced`         | Whether the prompt was enhanced by AI                |
| `enhanced_prompt`     | Enhanced version of the prompt (if any)              |

This metadata helps track how prompts evolve, what assistance was used, and how prompts relate to media.

In [5]:
chat_id = db.insert_chat("Example Chat", 1)

prompt1 = Prompt("Describe a futuristic city", depth=1)
prompt2 = Prompt("Add flying cars to the scene", depth=2)
db.save_prompt(prompt1, chat_id, user_id=1)
db.save_prompt(prompt2, chat_id, user_id=1)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


True

## 6. Inserting and Fetching Images

#### How Images Connect to Prompts:
Each image in the database is linked to prompts through:

* input_prompt_id: the prompt that inspired the image
* output_prompt_id: the prompt that describes the image (usually the same as input)
* prompt_guidance and image_guidance: numeric values guiding generation
* selected: whether the user chose this image as meaningful


#### How do we use this?
* Prompt/image connections allow tracing visual outputs to their origins.
* Guidance values reflect how strongly the prompt or image style influenced generation.
* The selected flag helps track which outputs users preferred or found useful.

In [9]:
db = Database()
db.connect()
img = Image.open("images/test_image.jpg")
prompt_image = PromptImage(image=img, prompt_guidance=7.0, image_guidance=0.4, 
                           input_prompt=prompt1.id, output_prompt=prompt1.id, save=True, selected=True)

db.save_image(prompt_image, session_id=chat_id, user_id=1)

# Fetch images
db = Database()
db.connect()
print(db.fetch_images_by_chat(chat_id))

                                     id                       input_prompt_id  \
0  d3047abb-39ef-4d61-9edc-7f05a31c0bb2  a1e3d379-0398-4900-acc9-16288bb875f0   
1  9ea1c308-ae8d-4643-ba72-4eaaae6b1a21  a1e3d379-0398-4900-acc9-16288bb875f0   
2  68248e2e-89ff-410e-b5ae-7ade0103a87b  a1e3d379-0398-4900-acc9-16288bb875f0   

                       output_prompt_id  user_id  chat_id  prompt_guidance  \
0  a1e3d379-0398-4900-acc9-16288bb875f0        1        2              7.0   
1  a1e3d379-0398-4900-acc9-16288bb875f0        1        2              7.0   
2  a1e3d379-0398-4900-acc9-16288bb875f0        1        2              7.0   

   image_guidance                                             path  selected  
0             0.4  images/d3047abb-39ef-4d61-9edc-7f05a31c0bb2.png         1  
1             0.4  images/9ea1c308-ae8d-4643-ba72-4eaaae6b1a21.png         1  
2             0.4  images/68248e2e-89ff-410e-b5ae-7ade0103a87b.png         1  


## 7. Exploring Metrics

| Metric Type       | Table Name              | What It Measures                                    |
| ----------------- | ----------------------- | --------------------------------------------------- |
| **BERTScore**     | `bertscore_metrics`     | Novelty of current prompt vs previous               |
| **Guidance**      | `guidance_metrics`      | Prompt/image guidance values used during generation |
| **LPIPS**         | `lpips_metrics`         | Visual change between images                        |
| **Functionality** | `functionality_metrics` | Suggestion/enhancement usage stats                  |
| **Prompt Words**  | `prompt_word_metrics`   | Word-level analysis of prompts                      |


In [ ]:
print("BERTScore:", db.fetch_bertscore_by_chat(chat_id))
print("LPIPS:", db.fetch_lpips_by_chat(chat_id))
print("Guidance:", db.fetch_guidance_by_chat(chat_id))
print("Functionality:", db.fetch_functionality_by_chat(chat_id))
print("Prompt Word Metrics:", db.fetch_prompt_word_metrics_by_chat(chat_id))

## 8. Resetting Tables

Explanation: Useful for clean testing.

In [ ]:
# db.reset_tables()  # Use only if you want to wipe everything


## 9. Advanced Use Case: Full Data Seeding

Below you will see how to simulate a full multi-chat, multi-prompt, multi-image scenario — basically reusing our code from `example.ipynb` as a real-world test dataset.



### As previously seen, we start with imports and checking the system path

In [10]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'app')))

print(sys.path)

['/usr/lib64/python39.zip', '/usr/lib64/python3.9', '/usr/lib64/python3.9/lib-dynload', '', '/home/scur0279/MMA/.venv/lib64/python3.9/site-packages', '/home/scur0279/MMA/.venv/lib/python3.9/site-packages', '/gpfs/home5/scur0279/MMA/app', '/scratch-local/62539/tmpsjhkpdhd', '/gpfs/home5/scur0279/MMA/app']


### We can then import the database class from `database.py`

In [ ]:
from db.database import Database
import uuid

### Making an example with 2 users, and 3 chats

In [ ]:
from modules.prompt import Prompt
from modules.prompt_image import PromptImage
from PIL import Image

# -------------------
# DB SETUP
# -------------------
db = Database()
db.connect()
db.reset_tables()
print("[DB RESET]")

# -------------------
# USERS
# -------------------
user_id_1 = db.insert_user("first_test_user", "password123")
user_id_2 = db.insert_user("second_test_user", "password456")
user_id_1 = 1
user_id_2 = 2
print(f"[USERS] user_1={user_id_1}, user_2={user_id_2}")

# -------------------
# CHAT 1: Simple perfume chain (user 1)
# -------------------
chat_1 = db.insert_chat("Perfume Prompt Chain", user_id_1)
print(f"[CHAT 1] id={chat_1}")

prompt1 = Prompt("Generate a bottle of perfume", depth=1)
prompt2 = Prompt("Generate a bottle of perfume in a desert", depth=2)
prompt3 = Prompt("Add sunset lighting to the perfume bottle scene", depth=3,
                 suggestion_used="Try sunset colors", modified_suggestion=False)
prompt3.is_enhanced = True
prompt3.enhanced_prompt = "A perfume bottle in a desert with glowing sunset hues"

for p in [prompt1, prompt2, prompt3]:
    db.save_prompt(p, chat_1, user_id=user_id_1)

image_paths = ["images/sauvage.png", "images/sauvage_desert.png", "images/sauvage_sunset.png"]
for path, prompt in zip(image_paths, [prompt1, prompt2, prompt3]):
    img = Image.open(path)
    pi = PromptImage(image=img, prompt_guidance=6.0, image_guidance=0.4,
                     input_prompt=prompt.id, output_prompt=prompt.id, save=True, selected=True)
    db.save_image(pi, session_id=chat_1, user_id=user_id_1)

db.connect()
# -------------------
# CHAT 2: Cyberpunk drone (user 1)
# -------------------
chat_2 = db.insert_chat("Futuristic Drone Chain", user_id_1)
print(f"[CHAT 2] id={chat_2}")

drone1 = Prompt("Design a futuristic drone", depth=1)
drone2 = Prompt("Add neon lighting to the drone", depth=2)
shared_drone_prompt = Prompt("Place the drone in a cyberpunk city", depth=3,
                             suggestion_used="Try placing it in a city", modified_suggestion=True)

# Save prompts
for p in [drone1, drone2, shared_drone_prompt]:
    db.save_prompt(p, chat_2, user_id=user_id_1)

# Add drone1 and drone2 images (selected)
img1 = Image.open("images/test_image.jpg")
pi1 = PromptImage(image=img1, prompt_guidance=7.0, image_guidance=0.5,
                  input_prompt=drone1.id, output_prompt=drone1.id, save=True, selected=True)
# pi1.selected = True
db.save_image(pi1, session_id=chat_2, user_id=user_id_1)

img2 = Image.open("images/test_image2.jpg")
pi2 = PromptImage(image=img2, prompt_guidance=7.0, image_guidance=0.5,
                  input_prompt=drone2.id, output_prompt=drone2.id, save=True, selected=True)
# pi2.selected = True
db.save_image(pi2, session_id=chat_2, user_id=user_id_1)

# Image 3: from shared prompt, not selected
img3 = Image.open("images/test_image3.jpg")
pi3 = PromptImage(image=img3, prompt_guidance=7.0, image_guidance=0.5,
                  input_prompt=shared_drone_prompt.id, output_prompt=shared_drone_prompt.id, save=True)
db.save_image(pi3, session_id=chat_2, user_id=user_id_1)

# Image 4: from same shared prompt, selected
img4 = Image.open("images/test_image4.jpg")
pi4 = PromptImage(image=img4, prompt_guidance=7.0, image_guidance=0.5,
                  input_prompt=shared_drone_prompt.id, output_prompt=shared_drone_prompt.id, save=True, selected=True)
# pi4.selected = True
db.save_image(pi4, session_id=chat_2, user_id=user_id_1)


# -------------------
# CHAT 3: Deep sci-fi chain (user 2)
# -------------------
db.connect()
chat_3 = db.insert_chat("Sci-Fi Fleet Chain", user_id_2)
print(f"[CHAT 3] id={chat_3}")

p1 = Prompt("A spaceship flying through clouds", depth=1)
p2 = Prompt("Add sunlight breaking through clouds", depth=2)
p3 = Prompt("Include a fleet of ships in the distance", depth=3)
p4 = Prompt("Make the ships look metallic", depth=4,
            suggestion_used="Make them metallic", modified_suggestion=False)
p5 = Prompt("Add reflections from the sun", depth=5)
p5.is_enhanced = True
p5.enhanced_prompt = "Fleet of metallic ships reflecting golden sunlight"

for p in [p1, p2, p3, p4, p5]:
    db.save_prompt(p, chat_3, user_id=user_id_2)

sci_images = [
    "images/cat.png",
    "images/cat1.png",
    "images/cat2.png",
    "images/cat3.png",
    "images/cat4.png"
]
guidances = [(6.0, 0.4), (6.5, 0.45), (7.0, 0.5), (7.5, 0.55), (8.0, 0.6)]

for path, (pg, ig), prompt in zip(sci_images, guidances, [p1, p2, p3, p4, p5]):
    img = Image.open(path)
    pi = PromptImage(image=img, prompt_guidance=pg, image_guidance=ig,
                     input_prompt=prompt.id, output_prompt=prompt.id, save=True, selected=True)
    pi.selected = True
    db.save_image(pi, session_id=chat_3, user_id=user_id_2)

# -------------------
# Done
# -------------------
db.close()
print("[DONE] Data seeded.")


[DB RESET]
[USERS] user_1=1, user_2=2
[CHAT 1] id=1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[CHAT 2] id=2


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[CHAT 3] id=3


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

[DONE] Data seeded.


### Next we can test all our fetch methods

In [ ]:
chat_id = 2
user_id = 1

def test_fetch_methods(chat_id, user_id):
    db = Database()
    db.connect()

    print("\n========== USERS ==========")
    print("All users:", db.fetch_all_users().shape)
    print(f"User by ID ({user_id}):\n", db.fetch_user_by_id(user_id))
    print("User by Username:\n", db.fetch_user_by_username("second_test_user"))

    print("\n========== CHATS ==========")
    print("All chats:", db.fetch_all_chats().shape)
    print(f"Chats by User ({user_id}):\n", db.fetch_chats_by_user(user_id))
    print(f"Chat by ID ({chat_id}):\n", db.fetch_chat_by_id(chat_id))

    print("\n========== PROMPTS ==========")
    print("All prompts:", db.fetch_all_prompts().shape)
    print(f"Prompts by Chat ({chat_id}):\n", db.fetch_prompts_by_chat(chat_id))
    print(f"Prompts by User ({user_id}):\n", db.fetch_prompts_by_user(user_id))

    # Use a known prompt ID from your inserted test data
    print("Prompt by ID:\n", db.fetch_prompt_by_id("1"))  # Replace "1" with real prompt id if needed

    print("\n========== IMAGES ==========")
    print("All images:", db.fetch_all_images().shape)
    print(f"Images by User ({user_id}):\n", db.fetch_images_by_user(user_id))
    print(f"Images by Chat ({chat_id}):\n", db.fetch_images_by_chat(chat_id))
    # doesn't work because need the uuid path name for the image
    print("Image by Path (images/e1a3fe54-f85d-4e86-a86f-9d076032f2e2.png):\n", db.fetch_image_by_path("images/e1a3fe54-f85d-4e86-a86f-9d076032f2e2.png"))

    # Replace with actual image_id from your test database
    sample_image_id = db.fetch_all_images().iloc[0]["id"]
    print(f"Image by ID ({sample_image_id}):\n", db.fetch_image_by_id(sample_image_id))

    print("\n========== METRICS ==========")

    print("All BERTScore metrics:\n", db.fetch_all_bertscore_metrics().shape)
    print(f"BERTScore by User ({user_id}):\n", db.fetch_bertscore_by_user(user_id))
    print(f"BERTScore by Chat ({chat_id}):\n", db.fetch_bertscore_by_chat(chat_id))

    print("\nGuidance metrics:", db.fetch_all_guidance_metrics().shape)
    print(f"Guidance by User ({user_id}):\n", db.fetch_guidance_by_user(1))

    print("\nLPIPS metrics:", db.fetch_all_lpips_metrics().shape)
    print(f"LPIPS by Chat ({chat_id}):\n", db.fetch_lpips_by_chat(chat_id))

    print("\nFunctionality metrics:", db.fetch_all_functionality_metrics().shape)
    print(f"Functionality by Chat ({chat_id}):\n", db.fetch_functionality_by_chat(chat_id))

    print("\nPrompt Word metrics:", db.fetch_all_prompt_word_metrics().shape)
    print(f"Prompt Word by Chat ({chat_id}):\n", db.fetch_prompt_word_metrics_by_chat(1))

    db.close()

# Run the tests
test_fetch_methods(chat_id, user_id)


========== USERS ==========
All users: (2, 3)
User by ID (1):
    id         username     password
0   1  first_test_user  password123
User by Username:
    id          username     password
0   2  second_test_user  password456

========== CHATS ==========
All chats: (3, 3)
Chats by User (1):
    id                   title  user_id
0   1    Perfume Prompt Chain        1
1   2  Futuristic Drone Chain        1
Chat by ID (2):
    id                   title  user_id
0   2  Futuristic Drone Chain        1

========== PROMPTS ==========
All prompts: (11, 12)
Prompts by Chat (2):
                                      id  user_id  chat_id  \
0  6e7b8d9d-6e45-4dc1-b3d3-d1bacd62f426        1        2   
1  9bff560a-0dc2-46ec-9999-433313c0c5be        1        2   
2  4645ffb5-78ed-4d7d-88c1-d60ee93c8115        1        2   

                                prompt  depth  used_suggestion  \
0            Design a futuristic drone      1                0   
1       Add neon lighting to the drone  

## 10. Conclusion


#### Summary: What the `Database` Class Enables

The `Database` class provides a structured, modular interface to manage and analyze user interactions within a multimodal AI system. Specifically, it enables:


##### **Core Functionality**

* **User Management**: Create, retrieve, and track users via `insert_user` and related methods.
* **Session Tracking**: Organize user interactions into **chats**, each containing a sequence of **prompts**.
* **Prompt Logging**: Save and retrieve prompts with full metadata such as depth, enhancement, suggestions, and associated media.


##### **Image Storage & Linkage**

* Save generated images along with their **input/output prompts**, **guidance values**, and **selection status**.
* Retrieve images by user, chat, or prompt context.


##### **Automated Metrics Tracking**

* **BERTScore**: Measures semantic novelty between prompt steps.
* **LPIPS**: Tracks visual difference between image generations.
* **Guidance Metrics**: Logs prompt/image guidance parameters.
* **Functionality Metrics**: Analyzes user behavior (e.g., suggestion use).
* **Prompt Word Metrics**: Extracts lexical features from prompt text.


##### **Evaluation & Research Support**

* All queries return **structured pandas DataFrames** for easy analysis.
* Built-in support for **data seeding, resetting**, and **full reproducibility**.
* Designed for use in Jupyter notebooks, visualization pipelines, or app backends.



#### Where and How the `Database` Class Is Used

The `Database` class serves as the **backend data layer** for the entire system. It can be used:

##### 1. **Jupyter Notebooks**

* For **data inspection**, **testing**, and **analysis**.
* As well as for creating "fake" data
* Example:

  ```python
  db = Database()
  db.connect()
  db.insert_user("test_user", "password123")
  prompts = db.fetch_prompts_by_user(user_id=1)
  ```


##### 2. **App Logic (Back End)**

* Inside your app's **controller or server code**, e.g.:

  * When a user submits a new prompt → `save_prompt(...)` is called.
  * When an image is generated → `save_image(...)` records it along with metrics.
* Used like this:

  ```python
  db.save_prompt(prompt_obj, chat_id=session_id, user_id=user_id)
  db.save_image(prompt_image_obj, session_id=session_id, user_id=user_id)
  ```


##### 3. **Metrics Collection**

* All metrics (e.g., BERTScore, LPIPS) are automatically calculated **inside** `save_prompt()` and `save_image()`.
* This ensures metrics are tied directly to user interactions without extra steps.



##### 4. **Data Exports or Reporting**

* Developers and researchers use the query methods like:

  ```python
  df = db.fetch_bertscore_by_chat(chat_id=1)
  ```

  to analyze trends, generate plots, or export CSVs for external tools.
